In [16]:
# default_exp validation

In [17]:
# hide
%reload_ext lab_black

# Geometry Validation

> geometry validations

In [18]:
# exporti
import logging
from abc import ABC, abstractmethod
from typing import Sequence, Union

import geopandas as gpd
from shapely.algorithms.cga import signed_area
from shapely.geometry.polygon import orient

logger = logging.getLogger(__name__)

In [19]:
# export


class Validator(ABC):
    def __init__(
        self, gdf: gpd.GeoDataFrame, add_new_column: bool = True, apply_fix: bool = True
    ):
        self.gdf = gdf
        self.add_new_column = add_new_column
        self.apply_fix = apply_fix

    @property
    @abstractmethod
    def validator_column_name(self):
        pass

    @abstractmethod
    def check(self, geometry):
        pass

    @abstractmethod
    def fix(self, geometry):
        pass

    def validate(self):
        gdf = self.gdf.copy()
        is_valid = gdf.geometry.apply(self.check)
        if self.add_new_column:
            gdf[self.validator_column_name] = is_valid
        if self.apply_fix:
            gdf.loc[~is_valid, "geometry"] = gdf[~is_valid].geometry.apply(self.fix)
        return gdf

In [20]:
# export
class OrientationValidator(Validator):
    validator_column_name = "is_oriented_properly"

    def check(self, geometry):
        return signed_area(geometry.exterior) >= 0

    def fix(self, geometry):
        return orient(geometry)

In [21]:
# export
class GeometryValidation:

    validators_map = {"orientation": OrientationValidator}

    def __init__(
        self,
        gdf: gpd.GeoDataFrame,
        validators: Sequence[Union[str, Validator]] = ("orientation",),
        add_validation_columns: bool = True,
        apply_fixes: bool = True,
    ) -> gpd.GeoDataFrame:
        self.gdf = gdf
        self.validators = validators
        self.add_validation_columns = add_validation_columns
        self.apply_fixes = apply_fixes

    def _get_validators(self):
        validators_classes = []
        for validator in self.validators:
            if isinstance(validator, str):
                if validator not in self.validators_map:
                    raise Exception("Invalid validator.")
                validator = self.validators_map[validator]
                validators_classes.append(validator)
            elif isinstance(validator, Validator):
                validators_classes.append(validator)
            else:
                raise Exception("Invalid validator.")
        return validators_classes

    def validate(self):
        validators = self._get_validators()
        gdf = self.gdf
        for validator in validators:
            gdf = validator(
                gdf,
                add_new_column=self.add_validation_columns,
                apply_fix=self.apply_fixes,
            ).validate()
        return gdf

In [22]:
# hide
from nbdev.export import notebook2script

notebook2script("00_validation.ipynb")

Converted 00_validation.ipynb.
